# Simple Substitution
`w266 Final Project: Crosslingual Word Embeddings`

The code in this notebook was used to develop an algorithm to generate crosslingual word embeddings by training on a monolingual corpus and substituting translations at runtime.

# Notebook Setup

In [1]:
# general imports
import os
import re
import sys
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import print_function
# tell matplotlib not to open a new window
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

In [2]:
# filepaths
BASE = '~/Documents/MIDS/w266/Data' #'/home/mmillervedam/Data'
PROJ = '~/Documents/MIDS/w266/FinalProject'#'/home/mmillervedam/ProjectRepo'
FPATH_EN = BASE + '/test/wiki_en_10K.txt' # first 10000 lines from wiki dump
FPATH_ES = BASE + '/test/wiki_es_10K.txt' # first 10000 lines from wiki dump
#FULL_EN = BASE + '/en/full.txt'
#FULL_ES = BASE + '/es/full.txt'
EN_ES_DICT = PROJ +'/XlingualEmb/data/dicts/en.es.panlex.all.processed'
EN_IT_DICT  = PROJ +'/XlingualEmb/data/dicts/en.it.panlex.all.processed'
EN_IT_RAW = PROJ + '/XlingualEmb/data/mono/en_it.shuf.10k'
EN_IT_RAW = "/Users/mmillervedam/Documents/MIDS/w266/FinalProject/XlingualEmb/data/mono/en_it.shuf.10k"

# Load & Preprocess Data
__`ORIGINAL AUTHORS SAY:`__ "Normally, the monolingual word embeddings are trained on billions of words. However, getting that much of monolingual data for a low-resource language is also challenging. That is why we only select the top 5 million sentences (around 100 million words) for each language." - _Section 5.1, Duong et. al._ 

In [3]:
from parsing import Corpus, Vocabulary, batch_generator

### Corpus

In [4]:
# load corpus
en_it_data = Corpus(EN_IT_RAW)

In [5]:
# Corpus Stats
!wc {EN_IT_RAW}

   20000  430928 3746786 /Users/mmillervedam/Documents/MIDS/w266/FinalProject/XlingualEmb/data/mono/en_it.shuf.10k


__`i.e.:`__ 20K sentences (10K in each language) with ~430K tokens
> So this must not be their full data For now, I'm just going to look at the top 20K words and see what happens. In reality we should probably modify the Vocab class so that it explicily collects the top words for each language separately and then concatenates the index.

### Dictionary

In [6]:
# loading english-italian dictionary
pld = pd.read_csv(EN_IT_DICT, sep='\t', names = ['en', 'it'], dtype=str)
en_set = set(pld.en.unique())
it_set = set(pld.it.unique())

In [7]:
# dictionary vocab lengths:
print('EN:', len(en_set))
print('IT:', len(it_set))

EN: 266450
IT: 258641


### Vocabulary

In [8]:
# train multilingual Vocabulary# create vocab
en_it_vocab = Vocabulary(en_it_data.gen_tokens(), size = 100000)

### CBOW Data Generator
__`CHECK PAPER for HYPERPARAMS!`__: I can't seem to find where they talk abou the context window size, embedding size and batch size they use -- it may actually be in the Vulic and Moens paper instead of the Duong one.

In [9]:
BATCH_SIZE = 48
WINDOW_SIZE = 8
MAX_EPOCHS = 1 # fail safe

In [10]:
batches = batch_generator(en_it_data, 
                          en_it_vocab, 
                          BATCH_SIZE, 
                          WINDOW_SIZE, 
                          MAX_EPOCHS)

In [17]:
# sanity check
for context, label in batches:
    print("CONTEXT IDS:", context[:5])
    print("LABEL IDS:", label[:5])
    break

CONTEXT IDS: [[21, 1832, 19747, 4, 8, 1713, 72, 340, 4, 3, 17108, 2269, 4, 9, 2604, 5], [1832, 19747, 4, 8, 1713, 72, 340, 2416, 3, 17108, 2269, 4, 9, 2604, 5, 1], [19747, 4, 8, 1713, 72, 340, 2416, 4, 17108, 2269, 4, 9, 2604, 5, 1], [4, 8, 1713, 72, 340, 2416, 4, 3, 2269, 4, 9, 2604, 5, 1], [8, 1713, 72, 340, 2416, 4, 3, 17108, 4, 9, 2604, 5, 1]]
LABEL IDS: [2416, 4, 3, 17108, 2269]
